In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')

if os.path.exists('/content/gdrive/My Drive/wheat/darknetdata/obj.zip'):
    !cp /content/gdrive/My\ Drive/wheat/darknetdata/obj.zip ../
    !mkdir ./data/obj
    !unzip ../obj.zip -d data/obj
    print("Data retrieved")

    !cp /content/gdrive/My\ Drive/wheat/darknetdata/cfg/highresnothresh.cfg ./cfg
    !cp /content/gdrive/My\ Drive/wheat/darknetdata/cfg/obj.names ./data
    !cp /content/gdrive/My\ Drive/wheat/darknetdata/cfg/highresnothresh.data ./data
    print("Config recieved")

    !cp /content/gdrive/My\ Drive/wheat/darknetdata/cfg/generate_train.py ./
    !python generate_train.py
    print("train.txt generated")

    !ls data #You should see a few jpg and txt files as well as train.txt

else: # Retrieve and zip dataset if it does not exist (since IO is slow in cloud compute, we must zip it.)
    import imutils
    import pandas as pd

    !pip install --upgrade --force-reinstall --no-deps kaggle

    drive.mount('/content/gdrive')
    !mkdir /root/.kaggle/
    !echo '{"username":"stanleyjzheng","key":"INSERTAPIKEYHERE"}' > /root/.kaggle/kaggle.json # For some reason, as of july 25, 2020, you need to copy this line onto a different cell to run

    %cd /content/gdrive/My Drive/wheat/input
    %pwd

    !kaggle competitions download -c global-wheat-detection
    !unzip \*.zip  && rm *.zip
    %cd /content

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137 # Download pretrained weights

Paste the following code into your console window and hit **Enter** to avoid timeouts
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

In [ ]:
!ln -s "/content/gdrive/My Drive/wheat/" '/simlinkwheat' # Simlink since obj.data does not allow spaces in file paths

In [ ]:
#!cp /content/gdrive/My\ Drive/wheat/darknetdata/backup/yolov4-obj_last.weights ./data
#Saves to /simlinkwheat/darknetdata/nothreshbackup
%%capture
!./darknet detector train ./data/highresnothresh.data ./cfg/highresnothresh.cfg /simlinkwheat/darknetdata/highresnothreshbackup/highresnothresh_last.weights -dont_show -map

In [ ]:
imShow('chart.png')

In [ ]:
!nvidia-smi

In [ ]:
!ls /simlinkwheat/darknetdata/backup/

In [ ]:
!./darknet detector map data/obj.data ./cfg/yolov4-objORIGINAL.cfg /simlinkwheat/darknetdata/backup/yolov4-objORIGINAL_6000.weights # Test

In [ ]:
!./darknet detector test data/obj.data ./cfg/yolov4-objORIGINAL.cfg /simlinkwheat/darknetdata/backup/yolov4-objORIGINAL_4000.weights /simlinkwheat/input/test/51b3e36ab.jpg -thresh 0.35 -dont_show # Show predictions
imShow('predictions.jpg')